In [6]:
# import libraries
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
pd.set_option('display.max_columns', 500)

# import plotly.figure_factory as ff
# import geopandas
# import shapely
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import numpy as np
# import seaborn as sns
from tqdm import tqdm
from sklearn.linear_model import LinearRegression, ElasticNet, Lasso, Ridge
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import  LabelEncoder
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error
# import xgboost as xgb
# import lightgbm as lgb

from sklearn.model_selection import TimeSeriesSplit

In [7]:
def SMAPE(a, f):
    return 1/len(a) * np.sum(2 * np.abs(f-a) / (np.abs(a) + np.abs(f))*100)

In [15]:
x1 =  np.array([5,1])
x1_r =  np.array([1/5])

pred1 = np.array([6/5])
pred1_r = np.array([6/5])

x2 = np.array([100,105])
x2_r = np.array([105/100])
pred2 = np.array([110])
pred2_r = np.array([110/100])


print('absolute', SMAPE(x1[[-1]],pred1) , SMAPE(x2[[-1]],pred2))
print('ratio', SMAPE(x1_r[[-1]],pred1_r) , SMAPE(x2_r[[-1]],pred2_r))


absolute 18.181818181818176 4.651162790697675
ratio 142.85714285714286 4.651162790697678


In [2]:
# load dataset
df_train = pd.read_csv('./data/train.csv')
df_test = pd.read_csv('./data/test.csv')
df_census = pd.read_csv('./data/census_starter.csv')

In [12]:
# combine train and testset
df_train['dataset'] = 'train'
df_test['dataset'] = 'test'
df = pd.concat((df_train, df_test)).sort_values('row_id').reset_index(drop=True)
df_all = df.merge(df_census, on = 'cfips', how='left')
df_all['first_day_of_month'] = pd.to_datetime(df_all["first_day_of_month"])
df_all["year"] = df_all["first_day_of_month"].dt.year
df_all['month'] = df_all["first_day_of_month"].dt.month

In [8]:
x = np.arange(6)
condlist = [x<3, x>3]
choicelist = [x, x**2]
np.select(condlist, choicelist, 42)

array([ 0,  1,  2, 42, 16, 25])

In [13]:
# add 2-year lag census data to each rows 

conditions = [df_all['year']==2019,df_all['year']==2020,df_all['year']==2021, df_all['year']==2022, df_all['year']==2023]
choices_bb = [df_all['pct_bb_2017'],df_all['pct_bb_2018'],df_all['pct_bb_2019'],df_all['pct_bb_2020'],df_all['pct_bb_2021'] ]
choices_college = [df_all['pct_college_2017'],df_all['pct_college_2018'],df_all['pct_college_2019'],
                   df_all['pct_college_2020'],df_all['pct_college_2021']]
choices_foreign = [df_all['pct_foreign_born_2017'],df_all['pct_foreign_born_2018'],df_all['pct_foreign_born_2019'],
                   df_all['pct_foreign_born_2020'],df_all['pct_foreign_born_2021']]
choices_workers = [df_all['pct_it_workers_2017'],df_all['pct_it_workers_2018'],df_all['pct_it_workers_2019'],
                   df_all['pct_it_workers_2020'],df_all['pct_it_workers_2021']]
choices_inc = [df_all['median_hh_inc_2017'],df_all['median_hh_inc_2018'],df_all['median_hh_inc_2019'],
               df_all['median_hh_inc_2020'],df_all['median_hh_inc_2021']]

In [19]:
df_all["pct_bb"] = np.select(conditions, choices_bb, default=np.nan)
df_all["pct_college"] = np.select(conditions, choices_college, default=np.nan)
df_all["pct_foreign"] = np.select(conditions, choices_foreign, default=np.nan)
df_all["pct_workers"] = np.select(conditions, choices_workers, default=np.nan)
df_all["pct_inc"] = np.select(conditions, choices_inc, default=np.nan)
# drop unnecessary columns
df_all.drop(['pct_bb_2017', 'pct_bb_2018',
       'pct_bb_2019', 'pct_bb_2020', 'pct_bb_2021', 'pct_college_2017',
       'pct_college_2018', 'pct_college_2019', 'pct_college_2020',
       'pct_college_2021', 'pct_foreign_born_2017', 'pct_foreign_born_2018',
       'pct_foreign_born_2019', 'pct_foreign_born_2020',
       'pct_foreign_born_2021', 'pct_it_workers_2017', 'pct_it_workers_2018',
       'pct_it_workers_2019', 'pct_it_workers_2020', 'pct_it_workers_2021',
       'median_hh_inc_2017', 'median_hh_inc_2018', 'median_hh_inc_2019',
       'median_hh_inc_2020', 'median_hh_inc_2021'], axis=1, inplace=True)

In [20]:
df_all

,row_id,cfips,county,state,first_day_of_month,microbusiness_density,active,dataset,year,month,pct_bb,pct_college,pct_foreign,pct_workers,pct_inc
0,10001_2019-08-01,10001,Kent County,Delaware,2019-08-01,4.995701,6624.0,train,2019,8,80.8,14.5,6.0,1.3,57647.0
1,10001_2019-09-01,10001,Kent County,Delaware,2019-09-01,5.031902,6672.0,train,2019,9,80.8,14.5,6.0,1.3,57647.0
2,10001_2019-10-01,10001,Kent County,Delaware,2019-10-01,5.098270,6760.0,train,2019,10,80.8,14.5,6.0,1.3,57647.0
3,10001_2019-11-01,10001,Kent County,Delaware,2019-11-01,5.124666,6795.0,train,2019,11,80.8,14.5,6.0,1.3,57647.0
4,10001_2019-12-01,10001,Kent County,Delaware,2019-12-01,5.086203,6744.0,train,2019,12,80.8,14.5,6.0,1.3,57647.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
147340,9015_2023-02-01,9015,NaN,NaN,2023-02-01,NaN,NaN,test,2023,2,87.1,13.9,4.6,1.4,71418.0
147341,9015_2023-03-01,9015,NaN,NaN,2023-03-01,NaN,NaN,test,2023,3,87.1,13.9,4.6,1.4,71418.0
147342,9015_2023-04-01,9015,NaN,NaN,2023-04-01,NaN,NaN,test,2023,4,87.1,13.9,4.6,1.4,71418.0
147343,9015_2023-05-01,9015,NaN,NaN,2023-05-01,NaN,NaN,test,2023,5,87.1,13.9,4.6,1.4,71418.0


In [22]:
# # let's see how '(2year-lag)pct_college' is related with micro-biz density
# import matplotlib.pyplot as plt

# test = df_all.set_index('first_day_of_month')
# test = test[test['cfips']==56001]
# fig = plt.figure(figsize = (12, 4))
# ax1 = fig.add_subplot(1,1,1)
# ax1.plot(test['microbusiness_density'], color='blue' , label='microbusiness_density')
# ax2 = ax1.twinx()
# ax2.plot(test['pct_college'], color='red' , label='college')

In [29]:
# # #
# train_pivoted = df_all.pivot(index='cfips',columns='first_day_of_month',values='microbusiness_density')
# plt.plot(train_pivoted.median(axis=0));
# plt.xticks(rotation=90);

In [35]:
# create function for MA and EWM
#SMA and EMA are both commonly-used trend indicators. SMA gives equal weight to all data points,  while EMA applies more weight to recent data points. 

def moving_average(df,i, n):
    MA = pd.Series(df[i].rolling(n, min_periods=n).mean(), name = 'MA_' + str(n))
    df = df.join(MA)
    return df

def weighted_moving_average(df,i, n):
    EMA = pd.Series(df[i].ewm(span=n, adjust=False, min_periods=n).mean(), name = 'EMA_' + str(n))
    df = df.join(EMA)
    return df

In [36]:
df_all = moving_average(df_all, 'microbusiness_density', 3)


In [70]:
x = df_all[df_all['cfips'] == 10001].head()[['first_day_of_month','microbusiness_density']]
x['x'] = x.rolling(2, min_periods=2)['microbusiness_density'].mean()
x['e_x'] = x.ewm(span=2, adjust=False, min_periods=2)['microbusiness_density'].mean()
x

,first_day_of_month,microbusiness_density,x,e_x
0,2019-08-01,4.995701,NaN,NaN
1,2019-09-01,5.031902,5.013802,5.019835
2,2019-10-01,5.098270,5.065086,5.072125
3,2019-11-01,5.124666,5.111468,5.107152
4,2019-12-01,5.086203,5.105435,5.093186


In [86]:
from sklearn.model_selection import TimeSeriesSplit
n_splits = 7
tscv = TimeSeriesSplit(n_splits)

In [88]:
sample = df_all[df_all['cfips']==9011]

In [92]:
# for fold, (train_index, test_index) in enumerate(tscv.split(sample[sample['dataset']=='train'])):
#     print("Fold: {}".format(fold))
#     print("TRAIN indices:", train_index, "\n", "TEST indices:", test_index)
#     print("\n")
#     # X_train, X_test = train_X.iloc[train_index], train_X.iloc[test_index]
#     # y_train, y_test = train_y.iloc[train_index], train_y.iloc[test_index]


def SMAPE(a, f):
    return 1/len(a) * np.sum(2 * np.abs(f-a) / (np.abs(a) + np.abs(f))*100)

In [93]:
def smape_cv(model):
    tscv = TimeSeriesSplit(n_splits=7)
    smape_list = []
    model_name = model.__class__.__name__
    for _, (train_index, test_index) in tqdm(enumerate(tscv.split(train_X), start=1), 
                                             desc=f'{model_name} Cross Validations', total=7):
        X_train, X_test = train_X.iloc[train_index], train_X.iloc[test_index]
        y_train, y_test = train_y.iloc[train_index], train_y.iloc[test_index]
        clf = model.fit(X_train, y_train)
        pred = clf.predict(X_test)
        smape = SMAPE(y_test, pred) 
        smape_list.append(smape)
    return model_name, smape_list


def print_smape_score(model):
    # print cv and save the average
    model_name, score = smape_cv(model)
    for i, r in enumerate(score, start=1):
        print(f'{i} FOLDS: {model_name} smape: {r:.4f}')
    print(f'\n{model_name} mean smape: {np.mean(score):.4f}')
    print('='*30)
    return model_name, np.mean(score)

In [ ]:
reg = LinearRegression(n_jobs=-1)
ridge = Ridge(alpha=0.8, random_state=1)

In [ ]:
models = []
scores = []
for model in [reg, ridge]:
    model_name, mean_score = print_smape_score(model)
    models.append(model_name)
    scores.append(mean_score)